# Read simulated results and save all together

In [ ]:
import pickle

for data in ['bank77', 'clinic', 'goemo' , 'massive_I', 'massive_D']:
    results_dict = {}
    for emb_type in ['distilbert', 'sbert', 'openai', 'e5-large']:
        
        results_dict[emb_type] = {}
        
        # Open the pickle file
        with open("sims_offline_results_" + emb_type + "_" + data + ".pkl", "rb") as file:
            cur_results = pickle.load(file)

        for seed in range(10):
            for method in ['kmeans','kmeadoids','spectral','agglomerative','gmm']:
                results_dict[emb_type][seed][method] = cur_results[emb_type][seed][method]

            for llm_type in ['gpt-4o','gpt-3.5-turbo','llama3.3-70b','deepseek-chat','claude-3-7-sonnet-20250219']:

                try:
                    results_dict[emb_type][seed][llm_type] = {}

                    for force_context_length in [0, 10]:
                        results_dict[emb_type][seed][llm_type][force_context_length] = {} 
                        
                        for max_llm_iter in [1, 5]:
                            results_dict[emb_type][seed][llm_type][max_llm_iter] = cur_results[emb_type][seed][method][force_context_length][max_llm_iter]
                
        with open("results_all_" + data + ".pkl", "wb") as f:
            pickle.dump(results_dict, f)


## Process all simulated results and calculate average and std for ACC, NMI, DIST

In [ ]:
import numpy as np
import pickle
results_summary = {}

for data in ['bank77', 'clinic', 'goemo' , 'massive_I', 'massive_D']:
    results_summary[data] = {}
    
    with open("results_all_" + data + ".pkl", "rb") as file:
        results_dict = pickle.load(file)

    
    for emb_type in ['distilbert', 'openai','e5-large', 'sbert']:
        results_summary[data][emb_type] = {}
        
        for method in ['k-means','k-medoids','Spectral','GMM','Agglomerative']:
            results_summary[data][emb_type][method] = {}
            acc = []
            nmi = []
            cen = []
            for seed in range(10):
                cur_results = results_dict[emb_type][seed][method]['results']
                acc.append(cur_results[0])
                nmi.append(cur_results[1])
                cen.append(cur_results[2])

            results_summary[data][emb_type][method]['average'] = list(np.round([100*np.mean(acc), 100*np.mean(nmi)],3)) + [np.round(np.mean(cen),3)]
            results_summary[data][emb_type][method]['std'] = list(np.round([100*np.std(acc), 100*np.std(nmi)],3)) + [np.round(np.std(cen),3)]

        for llm_type in ['gpt-3.5-turbo','gpt-4o','llama3.3-70b','deepseek-chat','claude-3-7-sonnet-20250219']:        results_summary[data][emb_type][llm_type] = {}
            for force in [0, 10]:
                for niter in [1,5]:
                    results_summary[data][emb_type][llm_type][str(force) + '-' + str(niter)] = {}
                    acc = []
                    nmi = []
                    cen = []
                    for seed in range(10):
                        cur_results = results_dict[emb_type][seed][llm_type][force][niter]['results']
                        acc.append(cur_results[0])
                        nmi.append(cur_results[1])
                        cen.append(cur_results[2])
                    results_summary[data][emb_type][llm_type][str(force) + '-' + str(niter)]['average'] = list(np.round([100*np.mean(acc), 100*np.mean(nmi)],1)) + [np.round(np.mean(cen),3)]
                    results_summary[data][emb_type][llm_type][str(force) + '-' + str(niter)]['std'] = list(np.round([100*np.std(acc), 100*np.std(nmi)],3)) + [np.round(np.std(cen),3)]

# Table including all traditional + all kllmeans with gpt-4o + all embeddings

In [ ]:
line = {}
emb_type = 'openai'
llm_type = 'gpt-4o'

for data in ['bank77','clinic', 'goemo', 'massive_D','massive_I']:
    line[data] = {}
    for emb_type in ['distilbert', 'openai','e5-large', 'sbert']:
        
        line[data][emb_type] = {}
        acc = {'average':[], 'std':[]}
        nmi = {'average':[], 'std':[]}
        cen = {'average':[], 'std':[]}
        method_names = []
        for method in ['0-1','0-5','10-1','10-5']:
            cur_results = results_summary[data][emb_type][llm_type][method]
            acc['average'].append(cur_results['average'][0])
            nmi['average'].append(cur_results['average'][1])
            cen['average'].append(cur_results['average'][2])
            acc['std'].append(cur_results['std'][0])
            nmi['std'].append(cur_results['std'][1])
            cen['std'].append(cur_results['std'][2])
            
            if method == '10-1':
                cur_method = '\\quad \\emph{FS-single}'
            elif method == '10-5':
                cur_method = '\\quad \\emph{FS-multiple}'
            elif method == '0-1':
                cur_method = '\\quad \\emph{single}'
            elif method == '0-5':
                cur_method = '\\quad \\emph{multiple}'
    
            method_names.append(cur_method)
       
        for other_method in ['kmeans','kmedoids','gmm','agglomerative','spectral']:
            cur_results = results_summary[data][emb_type][other_method]
    
            acc['average'].append(cur_results['average'][0])
            nmi['average'].append(cur_results['average'][1])
    
            cen['average'].append(cur_results['average'][2])
            acc['std'].append(cur_results['std'][0])
            nmi['std'].append(cur_results['std'][1])
            cen['std'].append(cur_results['std'][2])
    
            method_names.append(other_method)
    
        # Find the maximum value
        #val = max(acc['average'])
        #acc['average'] = [f"{{\\bf {x}}}" if x == val else x for x in acc['average']]
    
        #val = max(nmi['average'])
        #nmi['average'] = [f"{{\\bf {x}}}" if x == val else x for x in nmi['average']]
    
        #val = min(cen['average'])
        #cen['average'] = [f"{{\\bf {x}}}" if x == val else x for x in cen['average']]
        
        for ii in range(len(method_names)):
            cur_method = method_names[ii]
            line[data][emb_type][cur_method] = cur_method + '&' + str(acc['average'][ii]) + '(' + str(acc['std'][ii]) + ')&' + str(nmi['average'][ii]) + '(' + str(nmi['std'][ii]) + ')&' + str(cen['average'][ii]) + '(' + str(cen['std'][ii]) + ')'

for data in ['bank77','clinic', 'goemo', 'massive_D','massive_I']:
    print("-"*10)
    print(data)
    print("")
    text = ""
    for emb_type in ['distilbert', 'openai','e5-large', 'sbert']:
        text = text + '\\hline\n\\\\\n\\emph{' + emb_type + "}&&&\\\\\n"
        text = text + '\\hline\n'
        text = text + "k-LLMmeans&&&\\\\\n"
        for ii in range(len(method_names)):
            cur_method = method_names[ii]
            text = text + line[data][emb_type][cur_method] + '\\\\\n'
    print(text) 

In [ ]:
line[data][emb_type][cur_method]